In [142]:
import plotly
import pickle
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.display import display, HTML
from plotly.offline import init_notebook_mode, iplot

In [42]:
init_notebook_mode(connected=True)

In [70]:
########################
##    Reading data    ##
########################

data = pd.read_pickle('concat_data.pkl')
data.head()

,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,DATE,DIST_FROM_START,DEATHS_DIRECT_COST,DEATHS_INDIRECT_COST,INJURIES_DIRECT_COST,INJURIES_INDIRECT_COST
0,OKLAHOMA,Tornado,0,0,0,0,250000.0,0.0,1950-04-28,115,0.0,0.0,0.0,0.0
1,TEXAS,Tornado,0,0,0,0,25000.0,0.0,1950-04-29,116,0.0,0.0,0.0,0.0
2,PENNSYLVANIA,Tornado,2,0,0,0,25000.0,0.0,1950-07-05,183,0.0,0.0,30000.0,0.0
3,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0.0,1950-07-05,183,0.0,0.0,0.0,0.0
4,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0.0,1950-07-24,202,0.0,0.0,0.0,0.0


In [120]:
############################
##   States' short form   ##
############################

short_form = {
    'Alabama':        'AL', 'Alaska':       'AK', 'Arizona':      'AZ', 'Arkansas':       'AR',
    'California':     'CA', 'Colorado':     'CO', 'Connecticut':  'CT', 'Delaware':       'DE',
    'Florida':        'FL', 'Georgia':      'GA', 'Hawaii':       'HI', 'Idaho':          'ID',
    'Illinois':       'IL', 'Indiana':      'IN', 'Iowa':         'IA', 'Kansas':         'KS',
    'Kentucky':       'KY', 'Louisiana':    'LA', 'Maine':        'ME', 'Maryland':       'MD',
    'Massachusetts':  'MA', 'Michigan':     'MI', 'Minnesota':    'MN', 'Mississippi':    'MS',
    'Missouri':       'MO', 'Montana':      'MT', 'Nebraska':     'NE', 'Nevada':         'NV',
    'New Hampshire':  'NH', 'New Jersey':   'NJ', 'New Mexico':   'NM', 'New York':       'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio':         'OH', 'Oklahoma':       'OK',
    'Oregon':         'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota':   'SD', 'Tennessee':    'TN', 'Texas':        'TX', 'Utah':           'UT',
    'Vermont':        'VT', 'Virginia':     'VA', 'Washington':   'WA', 'West Virginia':  'WV',
    'Wisconsin':      'WI', 'Wyoming':      'WY'}

short_form = {str.upper(key): short_form[key] for key in short_form}
reverse = {short_form[key]: key for key in short_form}
print('All the states :', len(short_form))

All the states : 50


In [72]:
###############################################
##   Replacing state names with short ones   ##
###############################################

print('Before :', data.shape)
data.STATE = data.STATE.apply(lambda x: short_form[x] if x in short_form else None)
data = data.dropna()
print('After  :', data.shape)
data.head()

Before : (1533686, 14)
After  : (1497174, 14)


,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,DATE,DIST_FROM_START,DEATHS_DIRECT_COST,DEATHS_INDIRECT_COST,INJURIES_DIRECT_COST,INJURIES_INDIRECT_COST
0,OK,Tornado,0,0,0,0,250000.0,0.0,1950-04-28,115,0.0,0.0,0.0,0.0
1,TX,Tornado,0,0,0,0,25000.0,0.0,1950-04-29,116,0.0,0.0,0.0,0.0
2,PA,Tornado,2,0,0,0,25000.0,0.0,1950-07-05,183,0.0,0.0,30000.0,0.0
3,PA,Tornado,0,0,0,0,2500.0,0.0,1950-07-05,183,0.0,0.0,0.0,0.0
4,PA,Tornado,0,0,0,0,2500.0,0.0,1950-07-24,202,0.0,0.0,0.0,0.0


In [136]:
###############################
##      Making the plot      ##
###############################

years = list(map(str, range(data.DATE.min().year, data.DATE.max().year+1)))

figure = {'data': [],
          'layout': {'title': 'Event counts',
                     'geo': go.layout.Geo(scope='usa',
                                          projection=go.layout.geo.Projection(type='albers usa'),
                                          showlakes=True,
                                          lakecolor='rgb(255, 255, 255)')},
          'frames': []}

figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 40,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': min(years),
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

scl = [
    [0.0, 'rgb(242,240,247)'],
    [0.2, 'rgb(218,218,235)'],
    [0.4, 'rgb(188,189,220)'],
    [0.6, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]


states = data.STATE.unique()
year_data = data[data.DATE.apply(lambda x: x.year)==1950]
count_data = [[], []]
for state in states:
    count_data[0].append(state)
    count_data[1].append(year_data[year_data.STATE==state].shape[0])
figure['data'].append(go.Choropleth(colorscale = scl,
                                    autocolorscale = False,
                                    locations = count_data[0],
                                    z = count_data[1],
                                    locationmode = 'USA-states',
                                    text = [reverse[i] for i in count_data[0]],
                                    marker = go.choropleth.Marker(
                                        line = go.choropleth.marker.Line(
                                            color = 'rgb(255,255,255)',
                                            width = 2)),
                                    colorbar = go.choropleth.ColorBar(title = "Events")))

# make frames
for year in years:
    print(year, 'year is being processed...')
    year_data = data[data.DATE.apply(lambda x: x.year)==int(year)]
    count_data = [[], []]
    for state in states:
        count_data[0].append(state)
        count_data[1].append(year_data[year_data.STATE==state].shape[0])
    frame = {'data': [go.Choropleth(colorscale=scl,
                                    autocolorscale=False,
                                    locations=count_data[0],
                                    z=count_data[1],
                                    locationmode='USA-states',
                                    text=[reverse[i] for i in count_data[0]],
                                    marker=go.choropleth.Marker(
                                        line=go.choropleth.marker.Line(
                                            color='rgb(255,255,255)',
                                            width=2)),
                                    colorbar=go.choropleth.ColorBar(title="Events"))],
             'name': str(year)}
    
    figure['frames'].append(frame)
    slider_step = {'args': 
                   [[year],
                    {'frame': {'duration': 300, 'redraw': False},
                     'mode': 'immediate',
                     'transition': {'duration': 300}}],
                   'label': year,
                   'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]


1950 year is being processed...
1951 year is being processed...
1952 year is being processed...
1953 year is being processed...
1954 year is being processed...
1955 year is being processed...
1956 year is being processed...
1957 year is being processed...
1958 year is being processed...
1959 year is being processed...
1960 year is being processed...
1961 year is being processed...
1962 year is being processed...
1963 year is being processed...
1964 year is being processed...
1965 year is being processed...
1966 year is being processed...
1967 year is being processed...
1968 year is being processed...
1969 year is being processed...
1970 year is being processed...
1971 year is being processed...
1972 year is being processed...
1973 year is being processed...
1974 year is being processed...
1975 year is being processed...
1976 year is being processed...
1977 year is being processed...
1978 year is being processed...
1979 year is being processed...
1980 year is being processed...
1981 yea

In [143]:
#############################
##    Saving the figure    ##
#############################

with open('figure.pkl', 'wb') as file:
    pickle.dump(figure, file)